<a href="https://colab.research.google.com/github/Deelaw15/BMI-Calculator-Python/blob/main/Matrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2555840 sha256=147e14ee4913313ace437b4925de68ba94e38f61ab65f2eb0ebcf5c9db1c05f9
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [2]:
!pip install 'numpy<2'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
import pandas as pd
import os

In [3]:
!git clone https://github.com/Deelaw15/Movie-Recommendation-System.git

Cloning into 'Movie-Recommendation-System'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 76 (delta 35), reused 40 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 8.15 MiB | 12.38 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [4]:
os.chdir('/content/Movie-Recommendation-System/ml-latest-small')

In [5]:
!ls

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


In [6]:
train = pd.read_csv("ratings.csv")
train.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# Load your train data (from earlier step)
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)


In [8]:
# Split into train/test for evaluation
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [9]:
# Initialize SVD model
svd_model = SVD(n_factors=100, biased=True, random_state=42)

In [10]:
# Train model
svd_model.fit(trainset)

In [11]:
# Evaluate
from surprise import accuracy
predictions = svd_model.test(testset)
rmse = accuracy.rmse(predictions)
print(f"✅ SVD RMSE: {rmse:.4f}")

RMSE: 0.8807
✅ SVD RMSE: 0.8807


Fine tuning

In [12]:
from surprise.model_selection import GridSearchCV

In [13]:
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 40, 60],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.2]
}

grid = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
grid.fit(data)

In [14]:
print("✅ Best RMSE:", grid.best_score['rmse'])
print("🏆 Best Parameters:", grid.best_params['rmse'])

✅ Best RMSE: 0.8586289633084297
🏆 Best Parameters: {'n_factors': 150, 'n_epochs': 60, 'lr_all': 0.01, 'reg_all': 0.1}


In [15]:
best_svd = SVD(n_factors=150, n_epochs=60, lr_all=0.01, reg_all=0.1)
full_trainset = data.build_full_trainset()
best_svd.fit(full_trainset)

In [16]:
import joblib

joblib.dump(best_svd, r'd:\Data\Movie-Recommendation-System\best_svd_model.pkl')
print("✅ Best SVD model saved successfully!")

✅ Best SVD model saved successfully!
